In [1]:
# !pip install transformers==3.1.0

In [2]:
import pandas as pd

dataset_number = 3

# Lacafe
# text = 'txt'
# label = 'has_anger'
# train_data = pd.read_csv(f'./Datasets/Lacafe/df_dataset_train_{dataset_number}.csv')
# test_data = pd.read_csv(f'./Datasets/Lacafe/df_dataset_test_{dataset_number}.csv')

# Fortuna
# text = 'text'
# label = 'hatespeech_comb'
# train_data = pd.read_csv(f'./Datasets/Fortuna/2019-05-28_portuguese_hate_speech_binary_classification_train_{dataset_number}.csv')
# test_data = pd.read_csv(f'./Datasets/Fortuna/2019-05-28_portuguese_hate_speech_binary_classification_test_{dataset_number}.csv')

# Fortuna (undersampling)
# text = 'text'
# label = 'hatespeech_comb'
# train_data = pd.read_csv(f'./Datasets/FortunaUndersampling/2019-05-28_portuguese_hate_speech_binary_classification_undersampling_train_{dataset_number}.csv')
# test_data = pd.read_csv(f'./Datasets/FortunaUndersampling/2019-05-28_portuguese_hate_speech_binary_classification_undersampling_test_{dataset_number}.csv')

# Fortuna (Updated)
# text = 'text'
# label = 'hatespeech_comb'
# train_data = pd.read_csv(f'./Datasets/FortunaUpdated/2019-05-28_portuguese_hate_speech_binary_classification_train_{dataset_number}.csv')
# test_data = pd.read_csv(f'./Datasets/FortunaUpdated/2019-05-28_portuguese_hate_speech_binary_classification_test_{dataset_number}.csv')

# ToLD-BR
# text = 'text'
# label = 'hatespeech'
# train_data = pd.read_csv(f'./Datasets/ToldBR/ToLD-BR-Treated_train_{dataset_number}.csv')
# test_data = pd.read_csv(f'./Datasets/ToldBR/ToLD-BR-Treated_test_{dataset_number}.csv')

# ToLD-BR (undersampling)
# text = 'text'
# label = 'hatespeech'
# train_data = pd.read_csv(f'./Datasets/ToldBRUndersampling/ToLD-BR-Treated_undersampling_train_{dataset_number}.csv')
# test_data = pd.read_csv(f'./Datasets/ToldBRUndersampling/ToLD-BR-Treated_undersampling_test_{dataset_number}.csv')

# OffComBr
# text = 'text'
# label = 'offensive'
# train_data = pd.read_csv(f'./Datasets/OffComBR/OffComBR2_train_{dataset_number}.csv')
# test_data = pd.read_csv(f'./Datasets/OffComBR/OffComBR2_test_{dataset_number}.csv')

# HateBR
text = 'instagram_comments'
label = 'offensive_language'
train_data = pd.read_csv(f'./Datasets/HateBR/HateBR_train_{dataset_number}.csv')
test_data = pd.read_csv(f'./Datasets/HateBR/HateBR_test_{dataset_number}.csv')

# MixToldBRLacafe
# text = 'txt'
# label = 'offensive'
# train_data = pd.read_csv(f'./Datasets/MixToldBRLacafe/mix_toldbr_lacafe_train_{dataset_number}.csv')
# test_data = pd.read_csv(f'./Datasets/MixToldBRLacafe/mix_toldbr_lacafe_test_{dataset_number}.csv')

In [3]:
from datasets import Dataset, DatasetDict

print(train_data)
train_data = Dataset.from_pandas(train_data)
test_data = Dataset.from_pandas(test_data)

raw_datasets = DatasetDict({'train': train_data, 'test': test_data})

train_data
test_data
raw_datasets

2022-07-06 03:21:58.122150: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-06 03:21:58.122184: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


                                     instagram_comments  offensive_language
0     Mas o miss universo n coloca a mulher de manei...                   0
1                                Imbeciloide esse homi.                   1
2                    Viva Paulo Freire! Viva a educacão                   0
3     A perseguição a Lula ainda não acabou, a mídia...                   1
4     Bandido porque te tiraram da cadeia? Volta pra...                   1
...                                                 ...                 ...
4895                                 Parabéns, merecido                   0
4896  Ele não tem cacife para ser um representante d...                   1
4897             Quem foi o imbecil que esvreveu isso??                   1
4898                            Nojo desses vermes.....                   1
4899                                   Que exemplo !👏👏👏                   0

[4900 rows x 2 columns]


DatasetDict({
    train: Dataset({
        features: ['instagram_comments', 'offensive_language'],
        num_rows: 4900
    })
    test: Dataset({
        features: ['instagram_comments', 'offensive_language'],
        num_rows: 2100
    })
})

In [4]:
# Tratamento básico dos dados

train_texts = raw_datasets['train'][text]
train_labels = raw_datasets['train'][label]
test_texts = raw_datasets['test'][text]
test_labels = raw_datasets['test'][label]

print("TrainTexts Length: ", len(train_texts))
print("TrainLabels Length: ", len(train_labels))
print("TestTexts Length: ", len(test_texts))
print("TestLabels Length: ", len(test_labels))

TrainTexts Length:  4900
TrainLabels Length:  4900
TestTexts Length:  2100
TestLabels Length:  2100


In [5]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.1, stratify=train_labels)

In [6]:
print("TrainTexts Length: ", len(train_texts))
print("TrainLabels Length: ", len(train_labels))
print("ValidationTexts Length: ", len(val_texts))
print("ValidationLabels Length: ", len(val_labels))

TrainTexts Length:  4410
TrainLabels Length:  4410
ValidationTexts Length:  490
ValidationLabels Length:  490


In [7]:
print(sum(train_labels))
print(len(train_labels) - sum(train_labels))
print(sum(val_labels))
print(len(val_labels) - sum(val_labels))

2205
2205
245
245


In [8]:
print(test_labels[:15])

[0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1]


In [9]:
# # Tentando fazer o processo manualmente para resolver o problema de min e max length do tokenizador

# from transformers import AutoModelForSequenceClassification, AutoTokenizer
# from torch import tensor

# model = AutoModelForSequenceClassification.from_pretrained("neuralmind/bert-base-portuguese-cased", num_labels=2)
# tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')

# # premise = 'Vai tomar no cu. O presidente é um otário'
# premise = 'Você é um péssimo motorista, além de ter vindo daquela merda de lugar. Vai tomar no cu!'
# # candidate_labels = tensor([1]).unsqueeze(0)
# hypothesis = 'O exemplo é ofensivo.'

# # run through model pre-trained on MNLI
# input = tokenizer.encode(premise, hypothesis, return_tensors='pt', truncation_strategy='only_first')
# logits = model(input)
# print(logits)
# print(logits[:2][0])

# # we throw away "neutral" (dim 1) and take the probability of
# # "entailment" (2) as the probability of the label being true 
# entail_contradiction_logits = logits[:2][0]
# probs = entail_contradiction_logits.softmax(dim=1)
# print(probs)
# prob_label_is_true = probs[:,1]

# print(prob_label_is_true)

In [10]:
# Importando o Tokenizer

from transformers import pipeline, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', max_len=512)
# tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased', max_len=512)

classifier = pipeline("zero-shot-classification", model='neuralmind/bert-base-portuguese-cased', tokenizer=tokenizer)
# classifier = pipeline("zero-shot-classification", model='bert-base-multilingual-cased', tokenizer=tokenizer)

candidate_labels = ['não ofensivo', 'ofensivo']
#hypothesis_template = "Esse texto é {}"

results = classifier(test_texts, candidate_labels)#, hypothesis_template=hypothesis_template, min_length=5, max_length=512)

cm = {'true_positive': 0, 'true_negative': 0, 'false_positive': 0, 'false_negative': 0}
for i in range(len(results)):
    result = results[i]
    
    if result['labels'][0] == 'ofensivo':
        predicted_label = 1
    else:
        predicted_label = 0

    if (test_labels[i] == predicted_label):
        if test_labels[i] == 1:
            cm['true_positive'] += 1
        else:
            cm['true_negative'] += 1
    else:
        if test_labels[i] == 0:
            cm['false_positive'] += 1
        else:
            cm['false_negative'] += 1
        
print(cm)
precision = cm['true_positive'] / (cm['true_positive'] + cm['false_positive'])
recall = cm['true_positive'] / (cm['true_positive'] + cm['false_negative'])
fscore = 2 * ((precision * recall) / (precision + recall))
acertos = cm['true_positive'] + cm['true_negative']
total = len(test_data)
print(f'Acertos: {acertos}')
print(f'Total: {total}')
print(f'Accuracy: {str((acertos / total) * 100).replace(".", ",")}')
print(f'Precision: {str((precision * 100)).replace(".", ",")}')
print(f'Recall: {str((recall * 100)).replace(".", ",")}')
print(f'Fscore: {str((fscore * 100)).replace(".", ",")}')

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the

{'true_positive': 697, 'true_negative': 465, 'false_positive': 585, 'false_negative': 353}
Acertos: 1162
Total: 2100
Accuracy: 55,333333333333336
Precision: 54,36817472698908
Recall: 66,38095238095238
Fscore: 59,7770154373928
